This notebook contains the work to embed each article using GloVe with 300 dimensional vector representations for each word. Most of the work should be self-explanatory.

In [ ]:
import pandas as pd
%pip install torch
import torch

#This is the word2vec model and dependencies:

import numpy as np
# Get the interactive Tools for Matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA
#Needed for google collab
%pip install gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

#Some other packages used within the code:
import string
import unicodedata

Load in the GloVe files. You made need to do this slightly differently if not using Google Colab.

In [ ]:
from gensim.models import KeyedVectors
glove_file = datapath("/glove.6B.300d.txt")
word2vec_glove_file = get_tmpfile("glove.6B.300d.txt")
glove2word2vec(glove_file, word2vec_glove_file)

/tmp/ipython-input-3698418353.py:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, word2vec_glove_file)


(400000, 300)

In [ ]:
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

In [ ]:
df = pd.read_csv("article_event_templates.csv", encoding="latin1")
print(df.head())

   article_id                                         event_text
0           0  attain semblance ; be desire ; be group ; be s...
1           1  access license ; ai move ; build team ; certif...
2           2  2020 be ; actor cripple time ; actor move void...
3           3  adapt threat ; anticipate detect ; anticipate ...
4           4  apply patch ; be dependency ; charge waste ; c...


In [ ]:
groups = [df.event_text[0].split(";")]
groups[0]
len(df.event_text)

13075

This code is splitting each triple into it's actual words. This is just a template. I ran through everything after this code block.

In [ ]:
groups = [df.event_text[0].split(";")]
groups
def strip_unicode_punct(s: str) -> str:
    # remove any char whose Unicode category starts with "P" (punctuation)
    return "".join(ch for ch in s if not unicodedata.category(ch).startswith("P"))
vec_articles = []
for group in groups:
  vec_groups = []
  for triple in group:     # This is the actual triple for each sentence
    text = str(triple)
    raw_tokens = text.split()                   # 1) split first

    vec_triple = []
    for tok in raw_tokens:                      # 2) remove spaces per token
        cleaned = strip_unicode_punct(tok).casefold()
        if cleaned:                             # drop empties
            vec_triple.append(cleaned)
    vec_groups.append(vec_triple)
  vec_articles.append(vec_groups)



vec_articles


[[['attain', 'semblance'],
  ['be', 'desire'],
  ['be', 'group'],
  ['be', 'source'],
  ['benefit', 'be'],
  ['business', 'have', 'demand'],
  ['confront', 'threat'],
  ['content', 'help', 'retention'],
  ['defend', 'treachery'],
  ['demonstrate', 'compliance'],
  ['discussion', 'serve', 'result'],
  ['employee', 'converse', 'joke'],
  ['employee', 'make', 'decision'],
  ['episode', 'cover', 'topic'],
  ['episode', 'update', 'response'],
  ['feature', 'have', 'impact'],
  ['hacker', 'leverage', 'wild'],
  ['hacker', 'understand'],
  ['human', 'be'],
  ['irony', 'be', 'course'],
  ['keep', 'employee'],
  ['keep', 'secure'],
  ['know', 'maker'],
  ['landscape', 'change', 'act'],
  ['need', 'employee'],
  ['objective', 'be'],
  ['organization', 'have', 'guard'],
  ['organization', 'thrive', 'threat'],
  ['people', 'become', 'line'],
  ['practitioner', 'be', 'one'],
  ['program', 'design', 'core'],
  ['program', 'provide', 'look'],
  ['program', 'provide', 'reporting'],
  ['report', 'give'

In [ ]:
def strip_unicode_punct(s: str) -> str:
    # remove any char whose Unicode category starts with "P" (punctuation)
    return "".join(ch for ch in s if not unicodedata.category(ch).startswith("P"))

vec_articles = []
for article in df.event_text:
  groups = [g.strip() for g in str(article).split(";") if g.strip()]  # FIX: trim/skip empties
  article_groups = []  # FIX: collect groups per article (not per group)
  for group in groups:
    text = str(group)                  # FIX: don't iterate chars; use the group string
    raw_tokens = text.split()          # 1) split first

    vec_triple = []
    for tok in raw_tokens:             # 2) remove spaces per token
        cleaned = strip_unicode_punct(tok).casefold()
        if cleaned:                    # drop empties
          vec_triple.append(cleaned)
    article_groups.append(vec_triple)  # FIX: append to per-article list
  vec_articles.append(article_groups)  # FIX: append once per article

vec_articles[1][1]

['ai', 'move']

This is my attempt to actually encode all of the articles using GloVe. model[word] is the key to access the word encoding.

In [ ]:
for i, article in enumerate(vec_articles):          # article: list of groups
    for j, triple in enumerate(article):            # triple: list of tokens
        for k, word in enumerate(triple):           # word: str
            vec_articles[i][j][k] = (
                model[word] if word in model.key_to_index else None
            )


len(vec_articles)  ## It got all of the articles

13075

In [ ]:
len(vec_articles[0]) #Checking that it has the triples.


54

In [ ]:
df['encodings'] = vec_articles

In [ ]:
df.head()

,article_id,event_text,encodings
0,0,attain semblance ; be desire ; be group ; be s...,"[[[-0.20978, 0.62172, -0.39958, 0.04778, -0.02..."
1,1,access license ; ai move ; build team ; certif...,"[[[-0.3458, -0.49297, 0.16799, -0.45065, 0.247..."
2,2,2020 be ; actor cripple time ; actor move void...,"[[[-0.36381, -0.18182, 0.45282, 0.079841, -0.0..."
3,3,adapt threat ; anticipate detect ; anticipate ...,"[[[-0.43884, -0.29956, -0.46622, -0.17706, 0.1..."
4,4,apply patch ; be dependency ; charge waste ; c...,"[[[0.13898, -0.42025, -0.07672, 0.030195, 0.18..."


Now that I have the word2vecs, I am going to average among each triple. I think this could be the issue spot.

NOTE: I deleted the section of the code that adds "none" values. I believe this will fix the clustering issue.

In [ ]:
 article_means = []

 for article in df.encodings:
     triple_means = []
     for triple in article:
         #filtering out nons
         valid_vecs = [v for v in triple if isinstance(v, np.ndarray)]
         if not valid_vecs:
             #triple_means.append(None)
             continue

         #shape check: (num_words, 300)
         stacked = np.stack(valid_vecs, axis=0)

         #mean along rows
         triple_means.append(stacked.mean(axis=0))

     article_means.append(triple_means)

In [ ]:
print(len(article_means))
print(len(article_means[0]))
article_means[3000]

13075
54


[array([-0.13373001, -0.324067  , -0.10319666, -0.32943666,  0.25484765,
         0.18822931,  0.31777135,  0.27055633,  0.11735701, -1.9734    ,
         0.17582   ,  0.23323666,  0.10326967,  0.10004667, -0.22617666,
         0.05413667, -0.16535   , -0.08056   , -0.12035433, -0.07659932,
        -0.23169665,  0.18101366,  0.22319333,  0.357694  , -0.31645   ,
         0.15288998, -0.18165   ,  0.5511067 , -0.49803004,  0.18835334,
        -0.01692999,  0.11986867,  0.14275531,  0.18309967, -0.44698334,
        -0.05751333,  0.25704667,  0.104532  ,  0.37050363, -0.54039   ,
        -0.08592334,  0.01644333,  0.30226   ,  0.47855666, -0.19067132,
        -0.079395  , -0.02673773, -0.17235827,  0.03474666, -0.05532667,
         0.09429201,  0.23930065, -0.20258534, -0.21177667, -0.279198  ,
         0.247775  ,  0.19245666,  0.360454  , -0.10092666,  0.18268234,
        -0.17288333,  0.03502667,  0.07424167, -0.23587997,  0.29095   ,
         0.04830666,  0.05686333, -0.2737733 , -0.1

In [ ]:
df['triple_means'] = article_means

RAN INTO AN ISSUE: I need to split the data frame according to triple ids for each mean to actually use the clustering. I did that below here:

In [ ]:
import pandas as pd
import numpy as np



def expand_list_of_arrays(df, list_col, record_id_col=None):


    # Ensure the column is list-like
    df['triple_means'] = df['triple_means'].apply(
        lambda x: x if isinstance(x, (list, tuple, np.ndarray)) else []
    )

    # Explode the list into individual rows
    expanded = df.explode(list_col, ignore_index=True)

    # Add array index (order of array within original list)
    expanded["array_id"] = expanded.groupby("article_id").cumcount()

    # Rename the array column for clarity
    expanded = expanded.rename(columns={list_col: "array"})

    # Optional: ensure arrays are np.ndarrays
    expanded["array"] = expanded["array"].apply(
        lambda a: np.asarray(a) if not isinstance(a, np.ndarray) else a
    )

    return expanded[["article_id", "array_id", "array"]]



expanded_df = expand_list_of_arrays(df, list_col="triple_means", record_id_col='article_id')
expanded_df.columns=['article_id','triple_id','triple_means']
print(expanded_df.head())




   article_id  triple_id                                       triple_means
0           0          0  [-0.19743499, 0.30806655, -0.48925, 0.147035, ...
1           0          1  [-0.242715, 0.2413245, -0.022511, -0.212255, 0...
2           0          2  [-0.1297285, 0.556265, -0.217335, -0.0903095, ...
3           0          3  [-0.25955, 0.27256998, 0.16765499, -0.29671, 0...
4           0          4  [-0.31701, 0.297955, 0.118540004, -0.051825, 0...


Double checking the data type of the GloVe embeddings:

In [ ]:
print(type(expanded_df.triple_means[0]))
print(type(expanded_df.triple_means.values))

len(expanded_df.triple_means.values)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


502388

In [ ]:
# CONVERT TO ARRAYS IF NEEDED
vals = expanded_df["triple_means"].tolist()
[(i, type(v), np.asarray(v, dtype=object).shape) for i, v in enumerate(vals[:50])]
# look for shapes not equal to (384,) or types not ndarray/list


[(0, numpy.ndarray, (300,)),
 (1, numpy.ndarray, (300,)),
 (2, numpy.ndarray, (300,)),
 (3, numpy.ndarray, (300,)),
 (4, numpy.ndarray, (300,)),
 (5, numpy.ndarray, (300,)),
 (6, numpy.ndarray, (300,)),
 (7, numpy.ndarray, (300,)),
 (8, numpy.ndarray, (300,)),
 (9, numpy.ndarray, (300,)),
 (10, numpy.ndarray, (300,)),
 (11, numpy.ndarray, (300,)),
 (12, numpy.ndarray, (300,)),
 (13, numpy.ndarray, (300,)),
 (14, numpy.ndarray, (300,)),
 (15, numpy.ndarray, (300,)),
 (16, numpy.ndarray, (300,)),
 (17, numpy.ndarray, (300,)),
 (18, numpy.ndarray, (300,)),
 (19, numpy.ndarray, (300,)),
 (20, numpy.ndarray, (300,)),
 (21, numpy.ndarray, (300,)),
 (22, numpy.ndarray, (300,)),
 (23, numpy.ndarray, (300,)),
 (24, numpy.ndarray, (300,)),
 (25, numpy.ndarray, (300,)),
 (26, numpy.ndarray, (300,)),
 (27, numpy.ndarray, (300,)),
 (28, numpy.ndarray, (300,)),
 (29, numpy.ndarray, (300,)),
 (30, numpy.ndarray, (300,)),
 (31, numpy.ndarray, (300,)),
 (32, numpy.ndarray, (300,)),
 (33, numpy.ndarray,

Here I then averaged each triple mean over each article to get one 300d vector to represent each article.

In [ ]:
print(expanded_df.head())


def mean_vec(series):
    arrs = [np.asarray(x, dtype=float) for x in series if isinstance(x, np.ndarray)]
    return np.mean(np.vstack(arrs), axis=0)

article_means_df = (expanded_df
            .groupby("article_id")["triple_means"]
            .apply(mean_vec)
            .reset_index(name="article_glove_mean"))


#check that it is the length of how many articles we have
print(article_means_df.head())
len(article_means_df)
print(article_means_df.article_glove_mean[0])




   article_id  triple_id                                       triple_means
0           0          0  [-0.19743499, 0.30806655, -0.48925, 0.147035, ...
1           0          1  [-0.242715, 0.2413245, -0.022511, -0.212255, 0...
2           0          2  [-0.1297285, 0.556265, -0.217335, -0.0903095, ...
3           0          3  [-0.25955, 0.27256998, 0.16765499, -0.29671, 0...
4           0          4  [-0.31701, 0.297955, 0.118540004, -0.051825, 0...
   article_id                                 article_glove_mean
0           0  [-0.09620131652250334, 0.16020741358537366, 0....
1           1  [-0.06868020722642541, 0.14179533064365388, -0...
2           2  [0.0007957195048220456, 0.046490237739635634, ...
3           3  [-0.09352210153508739, 0.1971629415494726, -0....
4           4  [-0.10410228179812077, 0.11399390246896517, -0...
[-9.62013165e-02  1.60207414e-01  1.46886813e-02 -1.62351169e-01
 -4.51791508e-03  8.68133217e-02  8.75448291e-03  1.86017388e-02
  1.33121575e-02 -1.4464

In [ ]:
#export to csv
article_means_df.to_csv("dependency_glove.csv", index=False)


THIS IS THE END OF THE AVERAGING PROCESS! BELOW IS THE CLUSTERING.

**Please please please** only use article_means_df. This is the final data frame that should be considered.

In [ ]:
# Stack a list of 1D arrays into a 2D matrix for clustering
X = np.vstack(article_means_df['article_glove_mean'].values)
print(X.shape) # (num_chunks=502388, embedding_dim=300)
print(len(article_means_df))
print(len(article_means_df.article_glove_mean[0]))

(13075, 300)
13075
300


Normalize the embedding.

In [ ]:
from sklearn.preprocessing import normalize
# normalize embeddings to unit length
X_norm = normalize(X, norm='l2')

Set parameters

In [ ]:
# HDBSCAN Tuning Params
MIN_CLUSTER_SIZE = 3         # default=5
MIN_SAMPLES = 3               # default=None
CLUSTER_SELECTION_EPS = 0.05  # default=0.0

In [ ]:
# Define hdbscan clusterer
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=MIN_CLUSTER_SIZE,
    min_samples=MIN_SAMPLES,
    metric='euclidean', # default
    cluster_selection_epsilon=CLUSTER_SELECTION_EPS,
    cluster_selection_method='eom' # default
)

# fit model to data and predict cluster labels
cluster_labels = clusterer.fit_predict(X_norm)
# cluster_labels = clusterer.fit_predict(X_sample)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
# View the cluster counts
cluster_counts = (
    pd.Series(cluster_labels)
    .value_counts()
    .sort_index()
    .rename_axis("cluster")
    .reset_index(name="count")
)
cluster_counts.head(10)

,cluster,count
0,-1,3651
1,0,23
2,1,5
3,2,4
4,3,6
5,4,11
6,5,5
7,6,4
8,7,4
9,8,5


In [ ]:
#export to csv
#df.to_csv("dependency_word2vec.csv", index=False)